In [1]:
import os  
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3" 

import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D,GlobalAveragePooling2D, Dense, BatchNormalization, Activation
from keras.models import Model
from keras.layers import DepthwiseConv2D

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
USE_BN=True
DROPOUT=0.2 # keep 80%
CONCAT_AXIS=3
weight_decay=1e-4
DATA_FORMAT='channels_last' # Theano:'channels_first' Tensorflow:'channels_last'

log_filepath  = './mobilenet_slim_n_thinner'

Using TensorFlow backend.


In [2]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

In [3]:
def depthwise_separable(x,params):
    # f1/f2 filter size, s1 stride of conv
    (s1,f2) = params
    x = DepthwiseConv2D((3,3),strides=(s1[0],s1[0]), padding='same',
                        depthwise_initializer="he_normal",depthwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(int(f2[0]), (1,1), strides=(1,1), padding='same',
               kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    return x

In [4]:
def MobileNet(img_input,shallow=True, classes=10):
    """Instantiates the MobileNet.Network has two hyper-parameters
        which are the width of network (controlled by alpha)
        and input size.
        # Arguments
            alpha: optional parameter of the network to change the 
                width of model.
            shallow: optional parameter for making network smaller.
            classes: optional number of classes to classify images
                into.
    """
    # change stride
    x = Conv2D(int(32), (3,3), strides=(1,1), padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = depthwise_separable(x,params=[(1,),(64,)])
    x = depthwise_separable(x,params=[(1,),(128,)])# change stride
    x = depthwise_separable(x,params=[(1,),(128,)])
    x = depthwise_separable(x,params=[(1,),(256,)])# change stride
    x = depthwise_separable(x,params=[(1,),(256,)])
    x = depthwise_separable(x,params=[(2,),(512,)])
    if not shallow:
        for _ in range(5):
            x = depthwise_separable(x,params=[(1,),(512,)])
    x = depthwise_separable(x,params=[(2,),(1024,)])
    x = depthwise_separable(x,params=[(1,),(1024,)])

    x = GlobalAveragePooling2D()(x)
    out = Dense(classes, activation='softmax')(x)
    return out

In [5]:
img_input=Input(shape=(32,32,3))
output = MobileNet(img_input)
model=Model(img_input,output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 32, 32, 32)        320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
__________

In [6]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('mobilenet_slim_n_thinner.h5')

Epoch 1/300
782/782 [==============================] - 64s 82ms/step - loss: 2.0718 - acc: 0.4300 - val_loss: 2.0029 - val_acc: 0.4881
Epoch 2/300
782/782 [==============================] - 58s 74ms/step - loss: 1.6738 - acc: 0.5875 - val_loss: 1.7229 - val_acc: 0.5865
Epoch 3/300
782/782 [==============================] - 57s 73ms/step - loss: 1.4667 - acc: 0.6603 - val_loss: 1.4727 - val_acc: 0.6620
Epoch 4/300
782/782 [==============================] - 57s 73ms/step - loss: 1.3204 - acc: 0.7097 - val_loss: 1.2991 - val_acc: 0.7192
Epoch 5/300
782/782 [==============================] - 57s 73ms/step - loss: 1.2324 - acc: 0.7397 - val_loss: 1.2108 - val_acc: 0.7528
Epoch 6/300
782/782 [==============================] - 57s 73ms/step - loss: 1.1514 - acc: 0.7660 - val_loss: 1.1870 - val_acc: 0.7613
Epoch 7/300
782/782 [==============================] - 56s 72ms/step - loss: 1.0853 - acc: 0.7845 - val_loss: 1.2112 - val_acc: 0.7501
Epoch 8/300
782/782 [==============================] - 

782/782 [==============================] - 56s 72ms/step - loss: 0.3287 - acc: 0.9613 - val_loss: 0.6550 - val_acc: 0.8853
Epoch 62/300
782/782 [==============================] - 56s 72ms/step - loss: 0.3261 - acc: 0.9595 - val_loss: 0.6695 - val_acc: 0.8796
Epoch 63/300
782/782 [==============================] - 56s 72ms/step - loss: 0.3220 - acc: 0.9623 - val_loss: 0.6340 - val_acc: 0.8903
Epoch 64/300
782/782 [==============================] - 57s 73ms/step - loss: 0.3141 - acc: 0.9646 - val_loss: 0.6797 - val_acc: 0.8839
Epoch 65/300
782/782 [==============================] - 56s 72ms/step - loss: 0.3123 - acc: 0.9634 - val_loss: 0.7150 - val_acc: 0.8755
Epoch 66/300
782/782 [==============================] - 56s 71ms/step - loss: 0.3135 - acc: 0.9639 - val_loss: 0.6869 - val_acc: 0.8802
Epoch 67/300
782/782 [==============================] - 56s 72ms/step - loss: 0.3129 - acc: 0.9638 - val_loss: 0.6577 - val_acc: 0.88730.3127 - acc:
Epoch 68/300
782/782 [==========================

782/782 [==============================] - 56s 72ms/step - loss: 0.1739 - acc: 0.9986 - val_loss: 0.5199 - val_acc: 0.9155
Epoch 120/300
782/782 [==============================] - 56s 71ms/step - loss: 0.1728 - acc: 0.9989 - val_loss: 0.5233 - val_acc: 0.9141
Epoch 121/300
782/782 [==============================] - 57s 72ms/step - loss: 0.1724 - acc: 0.9986 - val_loss: 0.5296 - val_acc: 0.9134
Epoch 122/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1721 - acc: 0.9987 - val_loss: 0.5256 - val_acc: 0.9150
Epoch 123/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1714 - acc: 0.9985 - val_loss: 0.5185 - val_acc: 0.9173
Epoch 124/300
782/782 [==============================] - 56s 71ms/step - loss: 0.1706 - acc: 0.9988 - val_loss: 0.5239 - val_acc: 0.9154
Epoch 125/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1695 - acc: 0.9989 - val_loss: 0.5210 - val_acc: 0.9161
Epoch 126/300
782/782 [==============================] 

782/782 [==============================] - 56s 72ms/step - loss: 0.1422 - acc: 0.9994 - val_loss: 0.5178 - val_acc: 0.9180
Epoch 178/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1414 - acc: 0.9995 - val_loss: 0.5218 - val_acc: 0.9187
Epoch 179/300
782/782 [==============================] - 57s 72ms/step - loss: 0.1408 - acc: 0.9995 - val_loss: 0.5244 - val_acc: 0.9161
Epoch 180/300
782/782 [==============================] - 57s 72ms/step - loss: 0.1406 - acc: 0.9995 - val_loss: 0.5223 - val_acc: 0.9171
Epoch 181/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1404 - acc: 0.9994 - val_loss: 0.5185 - val_acc: 0.9172
Epoch 182/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1398 - acc: 0.9994 - val_loss: 0.5200 - val_acc: 0.9176
Epoch 183/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1395 - acc: 0.9995 - val_loss: 0.5228 - val_acc: 0.9169: 0s - loss: 0.1395 - acc: 0.99
Epoch 184/300
782/782 [=

782/782 [==============================] - 56s 72ms/step - loss: 0.1294 - acc: 0.9998 - val_loss: 0.5146 - val_acc: 0.9176
Epoch 237/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1298 - acc: 0.9997 - val_loss: 0.5130 - val_acc: 0.9170
Epoch 238/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1299 - acc: 0.9997 - val_loss: 0.5165 - val_acc: 0.9182
Epoch 239/300
782/782 [==============================] - 57s 73ms/step - loss: 0.1296 - acc: 0.9998 - val_loss: 0.5173 - val_acc: 0.9171ETA: 8s  - ETA: 1s - loss:
Epoch 240/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1296 - acc: 0.9996 - val_loss: 0.5104 - val_acc: 0.9192
Epoch 241/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1293 - acc: 0.9999 - val_loss: 0.5174 - val_acc: 0.9183
Epoch 242/300
782/782 [==============================] - 56s 72ms/step - loss: 0.1295 - acc: 0.9997 - val_loss: 0.5168 - val_acc: 0.9178
Epoch 243/300
782/782 [======

782/782 [==============================] - 57s 73ms/step - loss: 0.1275 - acc: 0.9997 - val_loss: 0.5155 - val_acc: 0.9190
Epoch 292/300
782/782 [==============================] - 57s 73ms/step - loss: 0.1276 - acc: 0.9996 - val_loss: 0.5125 - val_acc: 0.9192
Epoch 293/300
782/782 [==============================] - 57s 73ms/step - loss: 0.1275 - acc: 0.9997 - val_loss: 0.5127 - val_acc: 0.9187
Epoch 294/300
782/782 [==============================] - 57s 73ms/step - loss: 0.1274 - acc: 0.9996 - val_loss: 0.5183 - val_acc: 0.9183
Epoch 295/300
782/782 [==============================] - 57s 73ms/step - loss: 0.1274 - acc: 0.9998 - val_loss: 0.5152 - val_acc: 0.9183
Epoch 296/300
782/782 [==============================] - 57s 72ms/step - loss: 0.1271 - acc: 0.9998 - val_loss: 0.5087 - val_acc: 0.9186acc: 0.99
Epoch 297/300
782/782 [==============================] - 57s 73ms/step - loss: 0.1271 - acc: 0.9998 - val_loss: 0.5143 - val_acc: 0.9184: 1s - 
Epoch 298/300
782/782 [================